In [ ]:
from lsst.rsp import get_tap_service
import sqlite3
import pandas as pd

In [ ]:
service = get_tap_service("tap")

In [ ]:
db_fname = "adler_demo_testing_database_dp1.db"

In [ ]:
cnx = sqlite3.connect(db_fname)

In [ ]:
object_choice_query = f"""
SELECT * FROM dp1.SSObject ORDER BY numObs DESC
"""

object_choice_table = service.search(object_choice_query).to_table().to_pandas()

In [ ]:
object_choice_table

In [ ]:
# ssoid = 8268570668335894776
ssoid_list = (21163620217073748, 23133931615301680, 21163637482928473, 21164728252512342, 23133931615301681)

test_id = 23133931615301680
# 2^63 - 1 is the maximum signed 64-bit value that can be stored in SQLite hence only two extra zeros for the fake_id otherwise it's too big
# max__ = 9223372036854775807
fake_id = 2313393161530168000

In [ ]:
filter_name = "r"

In [ ]:
# diasource_sql_query = f"""
#             SELECT
#                 *
#             FROM
#                 dp03_catalogs_10yr.diaSource
#             WHERE
#                 ssObjectId = {ssoid}
#             """
diasource_sql_query = f"""
            SELECT
                *
            FROM
                dp1.DiaSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
diatable = service.search(diasource_sql_query).to_table().to_pandas()

In [ ]:
sssource_sql_query = f"""
            SELECT
                *
            FROM
                dp1.SSSource
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
sssource_table = service.search(sssource_sql_query).to_table().to_pandas()

In [ ]:
sssource_table.columns

In [ ]:
ssobject_sql_query = f"""
            SELECT
                *
            FROM
                dp1.SSObject
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
ssobject_table = service.search(ssobject_sql_query).to_table().to_pandas()

In [ ]:
ssobject_table.columns

In [ ]:
mpcorb_sql_query = f"""
            SELECT
                *
            FROM
                dp1.MPCORB
            WHERE
                ssObjectId in {ssoid_list}
            """

In [ ]:
mpcorb_table = service.search(mpcorb_sql_query).to_table().to_pandas()

In [ ]:
mpcorb_table

In [ ]:
# add a new fake object

In [ ]:
diatable[(diatable["ssObjectId"] == test_id)].value_counts("diaSourceId")

In [ ]:
diatable[(diatable["ssObjectId"] == test_id) & (diatable["band"] == "r")].value_counts("diaSourceId")

In [ ]:
df_dict = {"dia":diatable,
           "sss":sssource_table,
           "sso":ssobject_table,
           "mpc":mpcorb_table}

In [ ]:
for x in df_dict:

    _df = df_dict[x].copy()[df_dict[x]["ssObjectId"]==test_id]
    _df["ssObjectId"] = fake_id
    df_dict[x] = pd.concat([df_dict[x],_df],axis = 0).reset_index(drop=True)
    print(len(_df),len(df_dict[x]))

In [ ]:
diatable = df_dict["dia"]
sssource_table = df_dict["sss"]
ssobject_table = df_dict["sso"]
mpcorb_table = df_dict["mpc"]

In [ ]:
_df = diatable.copy()[diatable["ssObjectId"]==test_id]
_df["ssObjectId"] = fake_id
diatable = pd.concat([diatable,_df],axis = 0).reset_index(drop=True)
# print(len(_df),len(df_dict[x]))
_df

In [ ]:
import astropy.units as u
import numpy as np
def flux_to_magnitude(flux, flux_err=np.nan):
    """Converts a flux measurement (with units of nanoJanskys) and its associated error
    into AB magnitudes. If no flux error is provided, the returned magnitude error
    will be NaN.

    Parameters
    -----------
    flux : astropy.units.Quantity
        Flux value with units of nanoJanskys (u.nJy).

    flux_err : astropy.units.Quantity, optional
        Flux error with units of nanoJanskys (u.nJy). Default is np.nan (dimensionless),
        in which case the magnitude error will be returned as NaN.

    Returns
    -----------
    magnitude : float
        The flux converted into AB magnitude (unitless scalar).

    magnitude_err : float
        The propagated uncertainty in AB magnitude (unitless scalar).
        Returns NaN if flux_err is not provided.

    Notes
    -----------
    - This function assumes that `flux` and `flux_err` are `astropy.units.Quantity`
      objects in units of `u.nJy`.
    """
    # TODO Handle the masked arrays better here
    # (ideally I think we want to keep magnitude as a masked array rather than making magErr non-masked)
    magnitude = flux.to(u.ABmag).value
    magnitude_err = ((2.5 / np.log(10)) * (flux_err / flux)).value
    return magnitude, magnitude_err

def add_outburst_fluxspace(flux, mag_shift = -1.5):
    """
    Function for adding a synthetic outburst to a light curve in flux space as the DP1 DiaSource table does not contain magnitude.
    Converts the flux values to magnitude, applies the magnitude shift and then converts back to flux
    
    Parameters
    -----------
    flux : float or array-like
        The flux value(s) to apply the shift to
    mag_shift : float, optional
        The value in magnitude to shift the data by. Default: -1.5

    Returns
    -----------
    output_flux : float or array-like
        The flux value(s) with the magnitude shift applied
    """
    mag, *_ = flux_to_magnitude(flux * u.nJy)
    mag += mag_shift
    output_flux = (mag * u.ABmag).to(u.nJy).value
    return output_flux

In [ ]:
# add an outburst
t0 = 60648
t1 = 60653
mag_shift = -1.5

mask = ((diatable["ssObjectId"] == fake_id) &
        (diatable["midpointMjdTai"] > t0) &
        (diatable["midpointMjdTai"] < t1))

# DP1 DiaSource table does not contain magnitude so have to add the outburst in flux space
diatable.loc[mask,"apFlux"] = add_outburst_fluxspace(diatable.loc[mask, "apFlux"].to_numpy(), mag_shift = mag_shift)
diatable.loc[mask,"psfFlux"] = add_outburst_fluxspace(diatable.loc[mask, "psfFlux"].to_numpy(), mag_shift = mag_shift)
diatable.loc[mask,"scienceFlux"] = add_outburst_fluxspace(diatable.loc[mask, "scienceFlux"].to_numpy(), mag_shift = mag_shift)
diatable.loc[mask,"trailFlux"] = add_outburst_fluxspace(diatable.loc[mask, "trailFlux"].to_numpy(), mag_shift = mag_shift)

In [ ]:
# diatable.to_sql("DiaSource", con=cnx, if_exists="append", index=False)
diatable.to_sql("DiaSource", con=cnx, if_exists="replace", index=False)

In [ ]:
# sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)
sssource_table.to_sql("SSSource", con=cnx, if_exists="replace", index=False)

In [ ]:
# ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)
ssobject_table.to_sql("SSObject", con=cnx, if_exists="replace", index=False)

In [ ]:
# mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)
mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="replace", index=False)

In [ ]:
# _diatable = diatable.copy()[diatable["ssObjectId"] == test_id]
# _diatable["ssObjectId"] = fake_id
# _diatable["diaSourceId"] = _diatable["diaSourceId"] * 1000

# # add an outburst
# t0 = 63100
# t1 = 63600
# mag_shift = -1.5

# mask = (
#     (_diatable["ssObjectId"] == fake_id)
#     & (_diatable["midPointMjdTai"] > t0)
#     & (_diatable["midPointMjdTai"] < t1)
# )

# _diatable.loc[mask, "mag"] += mag_shift

# _diatable.to_sql("diaSource", con=cnx, if_exists="append", index=False)

In [ ]:
# _diatable[diatable["band"] == "r"]

In [ ]:
# _sssource_table = sssource_table.copy()[sssource_table["ssObjectId"] == test_id]
# _sssource_table["ssObjectId"] = fake_id
# _sssource_table["diaSourceId"] = _sssource_table["diaSourceId"] * 1000
# _sssource_table.to_sql("ssSource", con=cnx, if_exists="append", index=False)

# _ssobject_table = ssobject_table.copy()[ssobject_table["ssObjectId"] == test_id]
# _ssobject_table["ssObjectId"] = fake_id
# _ssobject_table.to_sql("ssObject", con=cnx, if_exists="append", index=False)

# _mpcorb_table = mpcorb_table.copy()[mpcorb_table["ssObjectId"] == test_id]
# _mpcorb_table["ssObjectId"] = fake_id
# _mpcorb_table.to_sql("MPCORB", con=cnx, if_exists="append", index=False)

In [ ]:
# cnx.close()

Testing everything went correctly...

In [ ]:
# cnx = sqlite3.connect(db_fname)

In [ ]:
# # example_query = f"""
# #                 SELECT
# #                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
# #                     topocentricDist, heliocentricDist
# #                 FROM
# #                     ssObject
# #                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
# #                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
# #                 WHERE
# #                     ssObject.ssObjectId = {ssoid}
# #                 """
# # example_query = f"""
# #                 SELECT
# #                     ssObject.ssObjectId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
# #                     topocentricDist, heliocentricDist
# #                 FROM
# #                     ssObject
# #                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
# #                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
# #                 WHERE
# #                     ssObject.ssObjectId in {ssoid_list}
# #                 """

# example_query = f"""
#                 SELECT
#                     ssObject.ssObjectId, ssSource.diaSourceId, mag, magErr, band, midpointMjdTai, ra, dec, phaseAngle,
#                     topocentricDist, heliocentricDist
#                 FROM
#                     ssObject
#                     JOIN diaSource ON ssObject.ssObjectId   = diaSource.ssObjectId
#                     JOIN ssSource  ON diaSource.diaSourceId = ssSource.diaSourceId
#                 WHERE
#                     ssObject.ssObjectId = {fake_id} and band = 'r'
#                 """

In [ ]:
# pd.read_sql_query(example_query, cnx)

In [ ]:
# cur = cnx.cursor()

In [ ]:
# res = cur.execute("SELECT * FROM sqlite_schema")

In [ ]:
# res.fetchall()